In [171]:
import pandas as pd
import numpy as np
import datetime as dt
import warnings
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
warnings.filterwarnings("ignore")
pd.set_option("mode.chained_assignment", None) 

In [3]:
raw_data = pd.read_csv("data/raw_data.csv")
data = raw_data[['optionid', 'date', 'exdate', 'cp_flag', 'strike_price', 'best_bid', 'best_offer', 'volume', 'open_interest', 'delta', 'gamma', 'vega', 'theta', 'impl_volatility']]
call_mask = data['cp_flag'] == 'C'
put_mask = data['cp_flag'] == 'P'
calls = data[call_mask]
puts = data[put_mask]
calls = calls.iloc[np.where(calls['impl_volatility'].notnull())]
calls.date = pd.to_datetime(calls['date'], format='%Y%m%d')
calls.exdate = pd.to_datetime(calls['exdate'], format='%Y%m%d')
calls['time_to_maturity'] = (calls.exdate - calls.date).dt.days

In [197]:
import yfinance as yf

sd = calls.date.iloc[0] - dt.timedelta(days=1)
ed = calls.date.iloc[-1] + dt.timedelta(days=1)
sp500 = yf.download('^GSPC',sd, ed)
sp500['date'] = sp500.index
sp500 = sp500[['date', 'Adj Close']]
sp500.columns = ['date', 'sp500']
sp500['sp500_return'] = sp500['sp500'] / sp500['sp500'].shift(1) - 1
sp500['sp500_log_return'] = np.log(sp500['sp500'] / sp500['sp500'].shift(1))
sp500 = sp500.iloc[1:]
sp500 = sp500[['date', 'sp500', 'sp500_log_return']]
sp500.columns = ['date', 'spot', 'spot_return']
# sp500_div = pd.read_csv("data/MULTPL-SP500_DIV_YIELD_MONTH.csv")
# sp500_div.Date = pd.to_datetime(sp500_div.Date)
vix = yf.download('^VIX',sd, ed)
vix = vix['Adj Close'].rename('VIX')
sp500_vix = sp500.join(vix)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [200]:
calls1 = calls.merge(sp500_vix, on = 'date')
calls1 = calls1[['optionid', 'date', 'time_to_maturity', 'strike_price', 'best_bid', 'best_offer', 'open_interest', 
                'delta', 'gamma', 'vega', 'theta', 'spot', 'spot_return', 'VIX', 'impl_volatility']]
calls1['ttm_yr'] = calls1.time_to_maturity / 252 # Validated with Table 1 that /252 instead of /365
calls1.strike_price /= 1000
df = calls1.copy()
df.sort_values(by=['optionid','date'], inplace=True)
df['optionid_chg'] = df.optionid.diff(-1) 
df['impvol_chg'] = (df.impl_volatility / df.impl_volatility.shift(1)) - 1
df = df[df.optionid_chg == 0]
df = df.iloc[1:]

In [202]:
df = df[(df.delta > 0.05) & (df.delta < 0.95)]
df = df[df.time_to_maturity > 14]
df.shape

(5426309, 18)

In [203]:
df_hw = df.loc[(df.date >= '2010-01-01') & (df.date <= '2017-12-31')]

In [204]:
# df_hw is original scale - df_display is for making up numbers only

df_display = df_hw[['spot_return', 'ttm_yr', 'delta', 'VIX', 'impvol_chg']].copy()
df_display.impvol_chg *= 10
df_display.spot_return *= 100
# impvol = (a/b-1) * 10
# spot_return = (a/b-1) * 100
df_display.describe().iloc[[1,2,3,-1]]

,spot_return,ttm_yr,delta,VIX,impvol_chg
mean,0.021137,0.807192,0.629758,15.883669,-0.069409
std,0.872940,0.964013,0.285287,5.410673,1.099345
min,-6.895837,0.059524,0.050001,9.140000,-9.681027
max,4.631744,4.345238,0.949999,48.000000,31.511413


In [177]:
# Trying the coef from paper

X1 = df_hw.spot_return / np.sqrt(df_hw.ttm_yr)
X2 = X1 * df_hw.delta
X3 = X1 * (df_hw.delta ** 2)

y = df_hw.impvol_chg.values

a = -0.2329
b = 0.4176
c = -0.4892

y_pred = a*X1 + b*X2 + c*X3

mean_squared_error(y_pred, y)

0.012017464486709305

In [178]:
# doing linear regression

X1 = df_hw.spot_return / np.sqrt(df_hw.ttm_yr)
X2 = X1 * df_hw.delta
X3 = X1 * df_hw.delta ** 2
X = pd.concat((X1,X2,X3), axis = 1).values
y = df_hw.impvol_chg.values
import statsmodels.api as sm
model = sm.OLS(y, X).fit()
print(model.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.031
Model:                            OLS   Adj. R-squared (uncentered):              0.031
Method:                 Least Squares   F-statistic:                          2.192e+04
Date:                Sun, 05 Dec 2021   Prob (F-statistic):                        0.00
Time:                        14:24:24   Log-Likelihood:                      1.6703e+06
No. Observations:             2081365   AIC:                                 -3.341e+06
Df Residuals:                 2081362   BIC:                                 -3.341e+06
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [208]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X, y)
print("R-squared is " + str(reg.score(X, y)))
y_pred = reg.predict(X)
print("MSE is " + str(mean_squared_error(y, y_pred)))

R-squared is 0.02424068661420775
MSE is 0.011792631698223003


In [206]:
df.to_csv('data/processed_all.csv')

In [207]:
df_hw.to_csv('data/processed_2010-2017.csv', index = False)

### Doing Train-test-split of 0.7/0.3

In [209]:
X = pd.concat((df_hw.spot_return,df_hw.time_to_maturity,df_hw.delta), axis = 1).values
y = df_hw.impvol_chg.values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [210]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X_train, y_train)
print("R-squared is " + str(reg.score(X_train, y_train)))
y_pred = reg.predict(X_test)
print("MSE is " + str(mean_squared_error(y_test, y_pred)))

R-squared is 0.024014680709512626
MSE is 0.011630220694000753


In [145]:
from sklearn.tree import DecisionTreeRegressor
reg = DecisionTreeRegressor(max_depth=5).fit(X_train, y_train)
print("R-squared is " + str(reg.score(X_train, y_train)))
y_pred = reg.predict(X_test)
print("MSE is " + str(mean_squared_error(y_test, y_pred)))

R-squared is 0.014896936420746298
MSE is 0.012665679111178368


In [146]:
from sklearn.ensemble import RandomForestRegressor
reg = RandomForestRegressor(max_depth=5).fit(X_train, y_train)
print("R-squared is " + str(reg.score(X_train, y_train)))
y_pred = reg.predict(X_test)
print("MSE is " + str(mean_squared_error(y_test, y_pred)))

KeyboardInterrupt: 

In [40]:
from sklearn.ensemble import GradientBoostingRegressor
reg = GradientBoostingRegressor(max_depth=5).fit(X, y)
print("R-squared is " + str(reg.score(X, y)))
y_pred = reg.predict(X)
print("MSE is " + str(mean_squared_error(y, y_pred)))

R-squared is 0.11487532410857226
MSE is 0.011380235766899177


ideas

SP500 Spot price/return (log) ---- T-1
T-1 implied Vol
remove last_date
T-1 best_bid best_offer (which implies implied vol)

paper model
LSTM / Transformer
Finance domain